In [1]:
import flair
from flair.data import Sentence, Token
from flair.models import TextClassifier

unable to import 'smart_open.gcs', disabling that module
I0411 14:20:40.658190 4590265792 file_utils.py:38] PyTorch version 1.2.0 available.


In [2]:
import string
import nltk
from segtok.segmenter import split_single

In [3]:
clf = TextClassifier.load('en-sentiment')

2020-04-11 14:20:43,948 loading file /Users/daniel/.flair/models/imdb-v0.4.pt


In [4]:
import newspaper
from newspaper import Article

In [20]:
import re
def preprocess(text):
    paragraphs = text.split("\n\n")
    topic_sentences = [para.split(".")[0] + "" for para in paragraphs]
    for i in range(len(topic_sentences)):
        topic_sentences[i] += "."
    x = " ".join(topic_sentences).translate(str.maketrans('', '', string.punctuation.replace(".","").replace("-",""))).replace("\n"," ")
    x = re.sub(' +', ' ',x)
    return x



In [21]:
def parse(url):
    article = Article(url)
    article.download()
    article.parse()
    t = article.text
    h = article.title
    return preprocess(t), preprocess(h)

In [22]:
def makeSentences(text):
    l = [Sentence(sent, use_tokenizer=True) for sent in split_single(text)]
        
    return l

In [23]:
def predict(url):
    text, headline = parse(url)
    t = makeSentences(text)
    h = makeSentences(headline)
    
   
    clf.predict(t)
    clf.predict(h)
    sent_t = getSentiment(t)
    sent_h = getSentiment(h)
    prediction = "Positive"
    if sent_t < 0 or sent_h < 0:
        prediction = "Negative"
        
    return {"Prediction":prediction, "Confidence":abs(sent_t+sent_h)/2}

In [24]:
def getSentiment(l):
    sent = 0
    for sentence in l:
        c = 1
        if sentence.labels[0].value == "NEGATIVE":
            c=-1
        sent += c*(sentence.labels[0].score)
    sent /= len(l)
    return sent

In [25]:
import pandas as pd
df_news = pd.read_csv("proofofconcept.csv")

In [26]:
urls = df_news['link']

In [43]:
import random
x = urls.tolist()
random.shuffle(x)
for url in x:
    a = Article(url)
    a.download()
    a.parse()
    score = predict(url)
    if score['Prediction'] == 'Positive' and "photograph" not in a.title.lower():
        print("Headline: " + a.title + " Score:" + str(score))

Headline: Childbirth in Venezuela, Where Women’s Deaths Are a State Secret Score:{'Prediction': 'Positive', 'Confidence': 0.629003381729126}
Headline: 6 hospitalized after shooting breaks out at California party Score:{'Prediction': 'Positive', 'Confidence': 0.5001093536615372}
Headline: Apple and Google Team Up to ‘Contact Trace’ the Coronavirus Score:{'Prediction': 'Positive', 'Confidence': 0.9328587651252747}
Headline: 7 Answers to Questions About the Malaria Drug Trump Keeps Pushing Score:{'Prediction': 'Positive', 'Confidence': 0.6198774576187134}
Headline: Coronavirus: Seven people stuck in a lift. Then one coughs... Score:{'Prediction': 'Positive', 'Confidence': 0.5499607920646667}
Headline: New York state now has more coronavirus cases than any country outside the US Score:{'Prediction': 'Positive', 'Confidence': 0.4878943582375844}
Headline: Chinese workers in the US are losing their visas. But flying home is too expensive Score:{'Prediction': 'Positive', 'Confidence': 0.53551

KeyboardInterrupt: 

In [49]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [64]:
def predict2(url):
    text, headline = parse(url)
    t = makeSentences2(text)
    h = makeSentences2(headline)
        
    sent_h = analyzer.polarity_scores(h)['compound']
    sent_t = 0
    for sent in t:
        sent_t += analyzer.polarity_scores(sent)['compound']
    sent_t /= len(t)

    prediction = "Positive"
    if sent_t < 0 or sent_h < 0:
        prediction = "Negative"
        
    return {"Prediction":prediction, "Confidence":abs(sent_t+sent_h)/2}

In [65]:
def makeSentences2(text):
    return [sent for sent in split_single(text)]

In [66]:
import textblob
from textblob import TextBlob

In [69]:
def predict3(url):
    text, headline = parse(url)
        
    sent_h = TextBlob(headline).sentiment.polarity
    sent_t = TextBlob(text).sentiment.polarity

    prediction = "Positive"
    if sent_t < 0 or sent_h < 0:
        prediction = "Negative"
        
    return {"Prediction":prediction, "Confidence":abs(sent_t+sent_h)/2}

In [85]:
x = urls.tolist()
def fetchNews(url_list):
    
    good_links = []
    for url in x:
        a = Article(url)
        try:
            a.download()
            a.parse()
            score_tb = predict3(url)
            if score_tb['Prediction'] == "Negative":
                score_tb['Confidence'] = -score_tb['Confidence']
            score_vader = predict2(url)
            if score_vader['Prediction'] == "Negative":
                score_vader['Confidence'] = -score_vader['Confidence']
            score = (score_vader['Confidence']+score_tb['Confidence'])/2

            if score > 0.25 and "photograph" not in a.title.lower():
                good_links.append([url, score])
        except:
            pass
    good_links = sorted(good_links,key=lambda x: x[1])
    return good_links

fetchNews(x)

[['https://www.theguardian.com/cities/2019/dec/28/secular-israel-sabbath-buses',
  0.2548013456937799],
 ['http://rss.cnn.com/~r/rss/cnn_world/~3/fkbbZuAKcjw/index.html',
  0.28247500000000003],
 ['https://www.bbc.co.uk/news/stories-52232551', 0.2992213068181818],
 ['https://abcnews.go.com/Live/video/abcnews-live-41463246',
  0.3132846590909091],
 ['http://rss.cnn.com/~r/rss/cnn_world/~3/I_WYjetrvmw/index.html',
  0.31486478575916377],
 ['https://www.cnbc.com/2020/04/10/who-sees-welcome-slowing-of-coronavirus-cases-in-hardest-hit-countries.html',
  0.32291151309008453],
 ['https://www.cnbc.com/2020/04/09/razer-min-liang-tan-gamer-mentality-behind-multimillion-dollar-brand.html',
  0.33458333333333334],
 ['https://www.nytimes.com/2020/04/09/world/middleeast/saudi-houthi-yemen-ceasefire-questions.html',
  0.38001108225108227],
 ['https://www.bbc.co.uk/news/world-52234125', 0.38791],
 ['http://rss.cnn.com/~r/rss/cnn_world/~3/Pa_SMoc1110/index.html',
  0.444980942669696],
 ['https://www.cn